In [33]:
import pandas as pd

In [34]:
dataset_path = "/Users/rohan/Trubbish/labeled_data.csv"

In [35]:
df = pd.read_csv(dataset_path)

In [36]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [52]:
import re

def clean_text(text):
    
    text = re.sub(r"http\S+", "", text)  
    text = re.sub(r"@\S+", "", text)     
    text = re.sub(r"#\S+", "", text)     
    text = re.sub(r"[^a-zA-Z\s]", "", text) 
    text = text.lower().strip()  
    return text

def remove_retweets(text):
   
    text = re.sub(r"^rt\s", "", text)  
    return text


df['cleaned_tweet'] = df['tweet'].apply(clean_text)


df['cleaned_tweet'] = df['cleaned_tweet'].apply(remove_retweets)


In [53]:
df['cleaned_tweet'].head()

0     as a woman you shouldnt complain about cleani...
1     boy dats coldtyga dwn bad for cuffin dat hoe ...
2     dawg rt  you ever fuck a bitch and she start ...
3                               she look like a tranny
4     the shit you hear about me might be true or i...
Name: cleaned_tweet, dtype: object

In [54]:
df['cleaned_tweet'].iloc[0]

' as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out'

In [58]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['cleaned_tweet'], df['class'], test_size=0.2, random_state=42
)


In [63]:
print(train_texts.head())

15272      well how else will white ppl get us to forge...
9351     funny thing isits not just the people doing it...
20323                    nigga messed with the wrong bitch
3638                                    bitch ass nigggaaa
20579                                   so that real bitch
Name: cleaned_tweet, dtype: object


In [73]:
import nltk
from nltk.tokenize import word_tokenize
import torch

# Ensure you have the NLTK punkt tokenizer downloaded
nltk.download('punkt')

def tokenize_function_nltk(texts):
    # Tokenize the list of texts (full texts, not individual tokens)
    tokenized_texts = [word_tokenize(text) for text in texts]
    
    # Padding to ensure all sequences have the same length
    max_length = 128
    padded_texts = [tokens[:max_length] + ['<pad>'] * (max_length - len(tokens)) if len(tokens) < max_length else tokens[:max_length] for tokens in tokenized_texts]
    
    # Convert tokenized texts to indices (you would typically use a dictionary here to map words to indices, but we'll skip that for now)
    # In this case, we'll map each word to its own index (for simplicity)
    vocab = {word: idx for idx, word in enumerate(set([token for text in padded_texts for token in text]))}
    
    token_ids = [[vocab[token] for token in text] for text in padded_texts]  # Convert tokens to indices
    
    # Convert the list of lists to a PyTorch tensor
    token_ids = torch.tensor(token_ids)  # Convert to torch tensors for later use
    return token_ids


train_encodings = tokenize_function_nltk(train_texts)
val_encodings = tokenize_function_nltk(val_texts)

print(train_encodings)


tensor([[2, 6, 1, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4],
        [3, 5, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4]])


[nltk_data] Downloading package punkt to /Users/rohan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
from torch.utils.data import Dataset

class CustomTextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        item = self.encodings[idx]
        return {"input_ids": item, "labels": item}  # Labels are the same as input for causal LM



In [75]:
train_dataset = CustomTextDataset(train_encodings)
val_dataset = CustomTextDataset(val_encodings)


In [76]:
pip install lightning


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 21.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [77]:
import pytorch_lightning as pl
from torch.optim import AdamW
from transformers import AutoModelForCausalLM

class LightningMistral(pl.LightningModule):
    def __init__(self, model_name, learning_rate=5e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def forward(self, input_ids, labels=None):
        # Forward pass through the model
        outputs = self.model(input_ids, labels=labels)
        return outputs

    def training_step(self, batch, batch_idx):
        # Training step
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        outputs = self(input_ids, labels=labels)
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Validation step
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        outputs = self(input_ids, labels=labels)
        loss = outputs.loss
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        # Optimizer configuration
        optimizer = AdamW(self.model.parameters(), lr=self.learning_rate)
        return optimizer


In [78]:
from torch.utils.data import DataLoader

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=8):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


In [79]:
# Model name (Mistral or any causal LM)
model_name = "mistralai/Mistral-7B"

# Instantiate model
model = LightningMistral(model_name=model_name, learning_rate=5e-5)

# Instantiate data module
data_module = CustomDataModule(train_dataset, val_dataset, batch_size=8)


OSError: mistralai/Mistral-7B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`